In [1]:
import numpy as np
import torch
from scripts.data_prep import create_dataset

In [2]:
input_path = "../data/UT_HAR_OG/input"
annotation_path = "../data/UT_HAR_OG/annotation"

In [3]:
data, labels = create_dataset(input_path, annotation_path)

In [9]:
# PHASE CALIBRATION
def calibrate_phase(data):
    proc_data = torch.zeros_like(data)
    num_antennas = 3
    num_subcarriers = 30

    for idx, tensor in enumerate(data):
        phase = tensor[:,num_antennas*num_subcarriers:]
        difference = torch.zeros_like(tensor[:,0])
        calibrated_phase = phase.clone()

        for i in range(num_subcarriers*num_antennas):
            if i%num_subcarriers == 0:
                  continue
            temp = phase[:,i] - phase[:,i-1]
            difference = difference + np.sign(temp) * (np.abs(temp) > np.pi)
            calibrated_phase[:,i] = phase[:,i] - difference * 2 * np.pi

        for i in range(num_antennas):
              k = (calibrated_phase[:,i*num_subcarriers +num_subcarriers-1] - calibrated_phase[:,i*num_subcarriers]) / (num_subcarriers-1)
              b = torch.mean(calibrated_phase[:,i*num_subcarriers:i*num_subcarriers+num_subcarriers], dim=1)

              for j in range(num_subcarriers):
                    calibrated_phase[:,i*num_subcarriers+j] -= k*j+b 

   

        
        proc_data[idx, :, :num_antennas*num_subcarriers] = tensor[:,:num_antennas*num_subcarriers]
        proc_data[idx, :, num_antennas*num_subcarriers:] = calibrated_phase

    return proc_data

In [ ]:
test_output = calibrate_phase(data)

In [ ]:
test_output.shape

In [ ]:
print(test_data[0,1,90:])

In [6]:
proc_data = torch.zeros_like(test_data)
num_antennas = 3
num_subcarriers = 30

In [ ]:
for idx, tensor in enumerate(test_data):
        phase = tensor[:,num_antennas*num_subcarriers:]
        difference = torch.zeros_like(tensor[:,0])
        calibrated_phase = phase.clone()

        for i in range(num_subcarriers*num_antennas):
            if i%num_subcarriers == 0:
                  continue
            temp = phase[:,i] - phase[:,i-1]
            difference = difference + np.sign(temp) * (np.abs(temp) > np.pi)
            calibrated_phase[:,i] = phase[:,i] - difference * 2 * np.pi

        for i in range(num_antennas):
              k = (calibrated_phase[:,i*num_subcarriers +num_subcarriers-1] - calibrated_phase[:,i*num_subcarriers]) / (num_subcarriers-1)
              b = torch.mean(calibrated_phase[:,i*num_subcarriers:i*num_subcarriers+num_subcarriers], dim=1)

              for j in range(num_subcarriers):
                    calibrated_phase[:,i*num_subcarriers+j] -= k*j+b 

        print(phase[0,30], phase[0,29])
        print(calibrated_phase[0])

In [13]:
import h5py
with h5py.File("../data/UT_HAR_CAL_PHASE/X.h5","w") as f:
    f.create_dataset("X",data=test_output)

: 